In [1]:
import warnings
warnings.filterwarnings("ignore")
import folium
import requests
import json
from pandas.io.json import json_normalize
import numpy as np
import pandas as pd

In [2]:
targetSite = 'https://www.bikeseoul.com/app/station/getStationRealtimeStatus.do'
req = requests.post(targetSite, data={
    'stationGrpSeq' : 'ALL'
})

In [3]:
# json 모듈의 loads() 함수로 크롤링한 json 타입의 문자열을 파이썬에서 사용하기 위해 딕셔너리 타입으로 변환한다.
bike_json = json.loads(req.text)
print(bike_json)

{'stationImgPath': '/nas_link/spb/attachFiles/file_admin/basePath', 'appUserSessionVO': {'usrSeq': None, 'encPwd': None, 'usrClsCd': None, 'usrDeviceId': None, 'snsType': None, 'voucherEndDttm': None, 'voucherSeq': None, 'loginId': None, 'mbId': None, 'mpnLostYn': None, 'lang': 'LAG_001', 'appOsType': None, 'regDttm': None, 'authCiVal': None, 'authClsCd': None, 'mbTelNo': None, 'mbEmailName': None, 'mbPostNo': None, 'mbAddr1': None, 'mbAddr2': None, 'parentSexCd': None, 'parentBirthDate': None, 'parentMpnNo': None, 'emailSendAgreeYn': None, 'lastLoginDttm': None, 'mbWgt': None, 'langClsCd': None, 'leaveYn': None, 'leaveReasonCd': None, 'leaveDttm': None, 'mbInfoColecAgreeDttm': None, 'mpnLostDttm': None, 'pagingYn': None, 'usrIp': None, 'partyVoucherSeq': None, 'elecVoucherSeq': None, 'requestSeq': None, 'usrDeviceType': None, 'snsId': None, 'usrType': None, 'viewFlg': 'list', 'usrBirthDate': None, 'sexCd': None, 'rentEncPwd': None, 'telecomClsCd': None, 'usrMpnNo': None, 'authDiVal': 

In [4]:
# pandas.io.json 모듈의 json_normalize() 함수로 json 타입의 데이터가 변환된 딕셔너리를 판다스 데이터프레임으로 변환한다.
# json_normalize(딕셔너리, '데이터프레임으로 변환할 데이터가 할당된 딕셔너리의 key')
bike_df = json_normalize(bike_json, ['realtimeList'])
bike_df

stationId stationImgFileName      stationName stationLongitude  \
0         ST-4                     102. 망원역 1번출구 앞     126.91062927   
1         ST-5                     103. 망원역 2번출구 앞     126.91083527   
2         ST-6                     104. 합정역 1번출구 앞     126.91498566   
3         ST-7                     105. 합정역 5번출구 앞     126.91482544   
4         ST-8                     106. 합정역 7번출구 앞     126.91282654   
...        ...                ...              ...              ...   
2691   ST-3170                       5861. 보라주유소 앞     126.92352295   
2692   ST-3187                      5862. 당산역11번출구     126.90145874   
2693   ST-3200                          5864.장훈고 앞     126.91397858   
2694   ST-3161                       6053. 중부세무서 앞     126.99066162   
2695   ST-1747                                상담센터     127.03565979   

     stationLatitude rackTotCnt parkingBikeTotCnt parkingQRBikeCnt  \
0        37.55564880         15                 0               24   
1        37.55495071         14                 0               25   
2        37.55062866         13                 0               11   
3        37.55000687          5                 0                3   
4        37.54864502         12                 0               11   
...              ...        ...               ...              ...   
2691     37.51319885         10                 0                4   
2692     37.53363037         14                 0                9   
2693     37.51156998          8                 0                8   
2694     37.56092453          5                 0               12   
2695     37.57164001          2                 0                1   

     parkingELECBikeCnt stationSeCd  mode  
0                     9     RAK_002  None  
1                     1     RAK_002  None  
2                     0     RAK_002  None  
3                     0     RAK_002  None  
4                     0     RAK_002  None  
...                 ...         ...   ...  
2691                  2     RAK_002  None  
2692                  1     RAK_002  None  
2693                  0     RAK_002  None  
2694                  1     RAK_002  None  
2695                  0     RAK_001  None  

[2696 rows x 11 columns]

In [5]:
# stationId: 대여소 id
# stationName: 대여소 이름
# stationLongitude: 대여소 경도
# stationLatitude: 대여소 위도
# rackTotCnt: 주차 가능한 자전거 대수
# parkingBikeToCnt: 주차된 따릉이 (LCD형) => 사용x
# parkingQRBikeCnt: 주차된 따릉이 (QR형) => 일반
# parkingELECBikeCnt: 주차된 새싹 따릉이 => 새싹

bike_df_map = bike_df[['stationId', 'stationName', 'stationLongitude', 'stationLatitude', 'parkingQRBikeCnt', 'parkingELECBikeCnt']]
bike_df_map

stationId      stationName stationLongitude stationLatitude  \
0         ST-4  102. 망원역 1번출구 앞     126.91062927     37.55564880   
1         ST-5  103. 망원역 2번출구 앞     126.91083527     37.55495071   
2         ST-6  104. 합정역 1번출구 앞     126.91498566     37.55062866   
3         ST-7  105. 합정역 5번출구 앞     126.91482544     37.55000687   
4         ST-8  106. 합정역 7번출구 앞     126.91282654     37.54864502   
...        ...              ...              ...             ...   
2691   ST-3170    5861. 보라주유소 앞     126.92352295     37.51319885   
2692   ST-3187   5862. 당산역11번출구     126.90145874     37.53363037   
2693   ST-3200       5864.장훈고 앞     126.91397858     37.51156998   
2694   ST-3161    6053. 중부세무서 앞     126.99066162     37.56092453   
2695   ST-1747             상담센터     127.03565979     37.57164001   

     parkingQRBikeCnt parkingELECBikeCnt  
0                  24                  9  
1                  25                  1  
2                  11                  0  
3                   3                  0  
4                  11                  0  
...               ...                ...  
2691                4                  2  
2692                9                  1  
2693                8                  0  
2694               12                  1  
2695                1                  0  

[2696 rows x 6 columns]

In [6]:
bike_df_map['stationLongitude'] = bike_df_map['stationLongitude'].astype('float')
bike_df_map['stationLatitude'] = bike_df_map['stationLatitude'].astype('float')
bike_df_map['parkingQRBikeCnt'] = bike_df_map['parkingQRBikeCnt'].astype('int')
bike_df_map['parkingELECBikeCnt'] = bike_df_map['parkingELECBikeCnt'].astype('int')

In [7]:
! pip install geopy

지오코딩(Geo Coding) => 주소로 위도, 경도 좌표 얻기
역지도코딩 => 위도, 경도 좌표로 주소 얻기

In [8]:
from geopy.geocoders import Nominatim

In [9]:
def geocoding(address):
    geolocoder = Nominatim(user_agent='South Korea', timeout=None)
    geo = geolocoder.geocode(address)
    return {'lat': str(geo.latitude), 'lot': str(geo.longitude)}

address = geocoding('서울 종로구 관철동')
print(address)

{'lat': '37.5739085', 'lot': '126.9821283'}


In [10]:
def geocoding_reverse(lat_lot):
    geolocoder = Nominatim(user_agent='South Korea', timeout=None)
    return geolocoder.reverse(lat_lot)

address = geocoding_reverse('37.5739085, 126.9821283')
print(address)

서울 조계사 백송, 우정국로, 견지동, 종로1·2·3·4가동, 종로구, 서울, 03150, 대한민국


In [11]:
bike_df_map['goo'] = np.NaN
bike_df_map['dong'] = np.NaN
bike_df_map

stationId      stationName  stationLongitude  stationLatitude  \
0         ST-4  102. 망원역 1번출구 앞        126.910629        37.555649   
1         ST-5  103. 망원역 2번출구 앞        126.910835        37.554951   
2         ST-6  104. 합정역 1번출구 앞        126.914986        37.550629   
3         ST-7  105. 합정역 5번출구 앞        126.914825        37.550007   
4         ST-8  106. 합정역 7번출구 앞        126.912827        37.548645   
...        ...              ...               ...              ...   
2691   ST-3170    5861. 보라주유소 앞        126.923523        37.513199   
2692   ST-3187   5862. 당산역11번출구        126.901459        37.533630   
2693   ST-3200       5864.장훈고 앞        126.913979        37.511570   
2694   ST-3161    6053. 중부세무서 앞        126.990662        37.560925   
2695   ST-1747             상담센터        127.035660        37.571640   

      parkingQRBikeCnt  parkingELECBikeCnt  goo  dong  
0                   24                   9  NaN   NaN  
1                   25                   1  NaN   NaN  
2                   11                   0  NaN   NaN  
3                    3                   0  NaN   NaN  
4                   11                   0  NaN   NaN  
...                ...                 ...  ...   ...  
2691                 4                   2  NaN   NaN  
2692                 9                   1  NaN   NaN  
2693                 8                   0  NaN   NaN  
2694                12                   1  NaN   NaN  
2695                 1                   0  NaN   NaN  

[2696 rows x 8 columns]

In [12]:
for i in range(bike_df_map.shape[0])[:5]:
    lat_lot = '{}, {}'.format(bike_df_map.loc[i, 'stationLatitude'], bike_df_map.loc[i, 'stationLongitude'])
    address = geocoding_reverse(lat_lot)
    addr = str(address).split(', ')
    print(addr[-4], addr[-5])

마포구 서교동
마포구 망원1동
마포구 서교동
마포구 서교동
마포구 합정동


In [13]:
bike_2022 = pd.read_csv('./data/bike_2022.csv', encoding='cp949')
bike_2022[['goo', 'dong']]

UnicodeDecodeError: 'cp949' codec can't decode byte 0xec in position 123: illegal multibyte sequence

In [ ]:
errorList = ['ST-212', 'ST-232', 'ST-339', 'ST-152', 'ST-196', 'ST-1771', 'ST-587', 'ST-2402', 'ST-1289', 'ST-1148', 'ST-1538', 'ST-664', 'ST-668', 'ST-936', 'ST-1224', 'ST-1241', 'ST-1521', 'ST-2352', 'ST-925', 'ST-926', 'ST-880', 'ST-881', 'ST-781', 'ST-890', 'ST-1168', 'ST-1311', 'ST-2416', 'ST-1861', 'ST-1753', 'ST-1947', 'ST-1948', 'ST-1949', 'ST-1950', 'ST-1824', 'ST-986', 'ST-2966', 'ST-2828', 'ST-2829', 'ST-2931', 'ST-2723', 'ST-2535', 'ST-2522', 'ST-3004']
for error in errorList:
    bike_error = bike_df_map[bike_df_map['stationId'] == error]
    lat_lot = '{}, {}'.format(bike_error.iloc[0, 3], bike_error.iloc[0, 2])
    print(lat_lot)


37.56829453, 126.91504669
37.56912231, 126.91479492
37.55134201, 126.90267181
37.56144714, 126.99311066
37.55853271, 127.00698853
37.56015396, 126.97410583
37.62009811, 127.10646057
37.66446304, 127.01184082
37.68859863, 127.05340576
37.46093369, 126.88716888
37.46055984, 126.88709259
37.50971985, 126.89009857
37.48493958, 126.90132141
37.48288345, 126.90103912
37.48160172, 126.89279938
37.4920311, 126.87377167
37.48485947, 126.89518738
37.50667191, 126.95777893
37.44765854, 127.05501556
37.45440674, 127.06492615
37.45623398, 127.01934814
37.4557991, 127.01628876
37.46750641, 127.03447723
37.46332169, 127.03604126
37.47438812, 127.03890228
37.45742416, 127.02265167
37.45924759, 127.02390289
37.477211, 127.13717651
37.48437119, 126.89542389
37.48062897, 126.82181549
37.48080826, 126.82190704
37.47790527, 126.82311249
37.47793961, 126.82327271
37.57666779, 126.95584869
37.55048752, 127.10895538
37.55302811, 127.18075562
37.48389053, 126.8221817
37.48337555, 126.82247925
37.45936584, 126.

In [ ]:
bike_df_map['goo'] = bike_2022['goo']
bike_df_map['dong'] = bike_2022['dong']
bike_df_map.head()

stationId      stationName  stationLongitude  ...  parkingELECBikeCnt  goo  dong
0      ST-4  102. 망원역 1번출구 앞        126.910629  ...                   8  마포구   서교동
1      ST-5  103. 망원역 2번출구 앞        126.910835  ...                   1  마포구  망원1동
2      ST-6  104. 합정역 1번출구 앞        126.914986  ...                   0  마포구   서교동
3      ST-7  105. 합정역 5번출구 앞        126.914825  ...                   0  마포구   서교동
4      ST-8  106. 합정역 7번출구 앞        126.912827  ...                   0  마포구   합정동

[5 rows x 8 columns]

In [ ]:
bike_df_map.groupby('goo').stationId.count()

goo
강남구        163
강동구        120
강북구         63
강서구        184
관악구         73
광명시          2
광진구         86
광희동          1
구로구        110
금천구         73
노원구        135
도봉구         77
동대문구        84
동작구         74
마포구        123
망우본동         1
서대문구        79
서초구        139
성동구         87
성북구         79
소공동          1
송파구        211
양천구        111
역삼2동         1
영등포구       144
용산구         93
은평구         98
종로5·6가동      1
종로구        103
중구          83
중랑구         92
천연동          1
하남시          3
Name: stationId, dtype: int64

In [21]:
bike_map = folium.Map(location=[bike_df_map['stationLatitude'].mean(), bike_df_map['stationLongitude'].mean()])

for index, data in bike_df_map.iterrows():
    station = data['stationName'].split(' ')
    if len(station) == 1:
        stationName = station[0]
    else:
        stationName = station[1].strip()
    string = '{}. 일반: {}대, 새싹: {}대'.format(stationName, data['parkingQRBikeCnt'], data['parkingELECBikeCnt'])
    popup = folium.Popup(string, max_width=300)
    folium.Marker(location=[data['stationLatitude'], data['stationLongitude']], popup=popup).add_to(bike_map)

bike_map.save('./output/bike.html')
bike_map